  **Final project is a tranlisterator from Hebrew to English, it contains an fst for the transliteration of the letters (the linguistics part) and some pre and post processing functions. The fst transliterates letter or diacritic to letter in state 0 with the exeption of letters that have certain diacritics (dagesh, shuruk, mapiq, shin/sin dot, shva) attached. The fst is tranduced with the function you wrote but with the print statements commented out since outputs were too long. There are two preprocessing functions and two post. Each function is explained in more detail above the function**
  
**At the bottom there are several print statements the run a Hebrew name or phrase through the "full_use" function. The "full_use" function calls the pre and post processing functions on the tranduce functions using the fst. Apostrophes in words are used to indicate that the letters don't lead into each other. For vowels this is saying pronounce one vowel and then the other, dont try to read them as one big vowel like in English. This ends up going a little overboard for the vowels, my names is transilerated as Ari'el which is an unecessary apostrophe, but I wanted in cases like the word leat which an American would read as leet otherwise but is actually le-at. This is also its own postprocessing function and can therefore be removed. For consonants it indicates a bit of a pause, it's not a full schwa sound, but it's also not an e and it's definitely not one letter read into the other (think br in break, it's not that). In most tranliterations it's written as an 'e' but I disagree with that. However it would be changing literally two apostrophes to 'e's if I decided that I liked that better.**

**All comments in the previous paragraph were styalistic choices that can easily be changed because they lead to names specifically looking sort of weird. However there were some problems with the letter vav because the way the they diacritics are typed people dont always use the right one (it looks right but unicode value is different) which leads to v being outputted as u. This is a rare occurance but vav is the only letter I've found that causes any problems and I think I fixed it by adding another state to my machine. Vav is also always v in consonant form never w, in Hebrew it can sometimes be read as w but it's one of those things where you just know when to change it and there are no actual indidcators.**


In [1]:
import re

#stolen from class
#transduce function
def n_transduce(f,s):
    #Parse string s with FST f and return its output
    curr_state = 0
    output = '' 
    for a in s:
        if (curr_state,a) in f:
            result = f[(curr_state,a)]
            #print(str(curr_state)+'->'+a+':'+result[1]+'->'+str(result[0]))
            output = output + result[1]
            curr_state = result[0]
        elif (curr_state,"@") in f: # for wildcard transitions
            result = f[(curr_state,"@")]
            #print(str(curr_state)+'->'+a+':'+a+'->'+str(result[0]))
            curr_state = result[0]
            output = output + result[1].replace("@",a)
        else:
            #print(str(curr_state)+'->'+a)
            return False
    if f[(curr_state,'')][0] == True:
        #print(str(curr_state)+"->'':"+f[(curr_state,'')][1])
        return output + f[(curr_state,'')][1]
    else:
        return False

In [2]:

#The giant fst for modern ashkenazi hebrew to english
#Most letters stay in state zero. Exceptions are letters that are changed by a dagesh, 
#shuruk/mapik, have shin/sin dot, and chiriq. 
#The fst keeps the shva vowel (two vertical dots under a letter) and in postprocessing the shva is
#either removed or replaced with an apostrophe depending on the placement in the word. 
hebrew_to_english = {
    
    (0, '\u05D0') : (0,''), #aleph
    (0, '\u05E2') : (0,''), #ayin
    
    (0, '\u05D1') : (4,''), #vet
    (0, '\uFB31') : (0,'b'), #bet
    (4, '\u05Bc') : (0,'b'), #dagesh
    
    (0, '\u05D2') : (0,'g'), #gimmel
    (0, '\u05D3') : (0,'d'), #daled
    (0, '\u05D4') : (0,'h'), #hey
    
    (0, '\u05D5') : (5,''), #vav
    (0, '\uFB35') : (0,'u'), #vav - shuruk
    (0, '\uFB4B') : (0,'o'), #vav - holam
    (5, '\u05BC') : (1,''), #dagesh
    (5, '\u05B9') : (0,'o'), #holam
    (5, '\u05BA') : (0,'o'), #holam haser
    
    (0, '\u05D6') : (0,'z'), #zayin
    (0, '\u05D7') : (0,'ch'), #chet - maybe change
    (0, '\u05D8') : (0,'t'), #tet
    (0, '\u05D9') : (0,'y'), #youd
    
    (0, '\u05DB') : (6,''), #chaf - maybe change
    (0, '\u05DA') : (6,''), #chaf sofit - maybe change
    (0, '\uFB3B') : (0,'k'), #kaf
    (0, '\uFB3A') : (0,'k'), #kaf sofit
    (6, '\u05BC') : (0,'k'), #dagesh
    
    (0, '\u05DC') : (0,'l'), #lamed
    
    (0, '\u05DE') : (0,'m'), #mem
    (0, '\u05DD') : (0,'m'), #mem sofit
    (0, '\u05E0') : (0,'n'), #nun
    (0, '\u05DF') : (0,'n'), #nun sofit
    
    (0, '\u05E1') : (0,'s'), #samech
    
    (0, '\u05E4') : (7,''), #feh
    (0, '\u05E3') : (7,''), #feh sofit
    (0, '\uFB44') : (0,'p'), #peh
    (0, '\uFB43') : (0,'p'), #peh sofit
    (7, '\u05BC') : (0,'p'), #dagesh
    
    (0, '\u05E6') : (0,'ts'), #tzadi
    (0, '\u05E5') : (0,'ts'), #tzadi sofit
    
    (0, '\u05E7') : (0,'q'), #kaf
    (0, '\u05E8') : (0,'r'), #resh
    
    (0, '\u05E9') : (3,'s'), #shin
    (0, '\uFB2A') : (0,'s'), #shin
    (0, '\uFB2B') : (0,'s'), #sin
    (3, '\u05C1') : (0,'h'), #shin
    (3, '\u05C2') : (0,''), #shin
    
    (0, '\u05EA') : (0,'t'), #tav
    
    (0, '\u05B7') : (0,'a'), #patach
    (0, '\u05B8') : (0,'a'), #kamatz
    (0, '\u05B2') : (0,'a'), #hataph patach
    
    (0, '\u05B1') : (0,'e'), #hataph segol
    (0, '\u05B6') : (0,'e'), #segol
    (0, '\u05B5') : (0,'e'), #tzerei
    
    (0, '\u05B4') : (2,'i'), #chiriq
    (0, '\u05BB') : (0,'u'), #kubutz
    (0, '\u05B0') : (0,'\u05B0'), #shva
    
    (0, '\u05B3') : (0,'o'), #hataph kamatz
    (0, '\u05B9') : (0,'o'), #holam chaser
    
    #stuff to deal with chiriq youd
    #(if a youd has a tzerei vowel before it we just want an i not yi)
    (2, '\u05D9') : (0,''), #youd
    (2, '\u05D0') : (0,''), #aleph
    (2, '\u05E2') : (0,''), #ayin
    (2, '\u05D1') : (4,''), #bet
    (2, '\uFB31') : (0,'b'), #vet
    (2, '\u05D2') : (0,'g'), #gimmel
    (2, '\u05D3') : (0,'d'), #daled
    (2, '\u05D4') : (0,'h'), #hey
    (2, '\u05D5') : (5,''), #vav
    (2, '\uFB35') : (0,'u'), #vav - shuruk
    (2, '\uFB4B') : (0,'o'), #vav - holam
    (2, '\u05D6') : (0,'z'), #zayin
    (2, '\u05D7') : (0,'ch'), #chet - maybe change
    (2, '\u05D8') : (0,'t'), #tet
    (2, '\u05DB') : (6,''), #chaf - maybe change
    (2, '\u05DA') : (6,''), #chaf sofit - maybe change
    (2, '\uFB3B') : (0,'k'), #kaf
    (2, '\uFB3A') : (0,'k'), #kaf sofit
    (2, '\u05DC') : (0,'l'), #lamed
    (2, '\u05DE') : (0,'m'), #mem
    (2, '\u05DD') : (0,'m'), #mem sofit
    (2, '\u05E0') : (0,'n'), #nun
    (2, '\u05DF') : (0,'n'), #nun sofit
    (2, '\u05E1') : (0,'s'), #samech
    (2, '\u05E4') : (7,''), #feh
    (2, '\u05E3') : (7,''), #feh sofit
    (2, '\uFB44') : (0,'p'), #peh
    (2, '\uFB43') : (0,'p'), #peh sofit
    (2, '\u05E6') : (0,'ts'), #tzadi
    (2, '\u05E5') : (0,'ts'), #tzadi sofit
    (2, '\u05E7') : (0,'q'), #kaf
    (2, '\u05E8') : (0,'r'), #resh
    (2, '\u05E9') : (3,'s'), #shin
    (2, '\uFB2A') : (0,'s'), #shin
    (2, '\uFB2B') : (0,'s'), #sin
    (2, '\u05EA') : (0,'t'), #tav
    
    #stuff to deal with shin-sin
    #assume the letter is a sin if not follwed by the shin dot
    (3, '\u05D0') : (0,''), #aleph
    (3, '\u05E2') : (0,''), #ayin
    (3, '\u05D1') : (4,''), #bet
    (3, '\uFB31') : (0,'b'), #vet
    (3, '\u05D2') : (0,'g'), #gimmel
    (3, '\u05D3') : (0,'d'), #daled
    (3, '\u05D4') : (0,'h'), #hey
    (3, '\u05D5') : (5,''), #vav
    (3, '\uFB35') : (0,'u'), #vav - shuruk
    (3, '\uFB4B') : (0,'o'), #vav - holam
    (3, '\u05D6') : (0,'z'), #zayin
    (3, '\u05D7') : (0,'ch'), #chet - maybe change
    (3, '\u05D8') : (0,'t'), #tet
    (3, '\u05D9') : (0,'y'), #youd
    (3, '\u05DB') : (6,''), #chaf - maybe change
    (3, '\u05DA') : (6,''), #chaf sofit - maybe change
    (3, '\uFB3B') : (0,'k'), #kaf
    (3, '\uFB3A') : (0,'k'), #kaf sofit
    (3, '\u05DC') : (0,'l'), #lamed
    (3, '\u05DE') : (0,'m'), #mem
    (3, '\u05DD') : (0,'m'), #mem sofit
    (3, '\u05E0') : (0,'n'), #nun
    (3, '\u05DF') : (0,'n'), #nun sofit
    (3, '\u05E1') : (0,'s'), #samech
    (3, '\u05E4') : (7,''), #feh
    (3, '\u05E3') : (7,''), #feh sofit
    (3, '\uFB44') : (0,'p'), #peh
    (3, '\uFB43') : (0,'p'), #peh sofit
    (3, '\u05E6') : (0,'ts'), #tzadi
    (3, '\u05E5') : (0,'ts'), #tzadi sofit
    (3, '\u05E7') : (0,'q'), #kaf
    (3, '\u05E8') : (0,'r'), #resh
    (3, '\u05E9') : (3,'s'), #shin
    (3, '\uFB2A') : (0,'s'), #shin
    (3, '\uFB2B') : (0,'s'), #sin
    (3, '\u05EA') : (0,'t'), #tav
    (3, '\u05B7') : (0,'a'), #patach
    (3, '\u05B8') : (0,'a'), #kamatz
    (3, '\u05B2') : (0,'a'), #hataph patach
    (3, '\u05B1') : (0,'e'), #hataph segol
    (3, '\u05B6') : (0,'e'), #segol
    (3, '\u05B5') : (0,'e'), #tzerei
    (3, '\u05B4') : (2,'i'), #chiriq
    (3, '\u05BB') : (0,'u'), #kubutz
    (3, '\u05B0') : (0,'\u05B0'), #shva
    (3, '\u05B3') : (0,'o'), #hataph kamatz
    (3, '\u05B9') : (0,'o'), #holam chaser
    
    #stuff to deal with bet vet
    #assume its a vet unless followed by a dagesh
    (4, '\u05D0') : (0,'v'), #aleph
    (4, '\u05E2') : (0,'v'), #ayin
    (4, '\u05D1') : (4,''), #bet
    (4, '\uFB31') : (0,'b'), #vet
    (4, '\u05D2') : (0,'vg'), #gimmel
    (4, '\u05D3') : (0,'vd'), #daled
    (4, '\u05D4') : (0,'vh'), #hey
    (4, '\u05D5') : (5,'v'), #vav
    (4, '\uFB35') : (0,'vu'), #vav - shuruk
    (4, '\uFB4B') : (0,'vo'), #vav - holam
    (4, '\u05D6') : (0,'vz'), #zayin
    (4, '\u05D7') : (0,'vch'), #chet - maybe change
    (4, '\u05D8') : (0,'vt'), #tet
    (4, '\u05D9') : (0,'vy'), #youd
    (4, '\u05DB') : (6,'v'), #chaf - maybe change
    (4, '\u05DA') : (6,'v'), #chaf sofit - maybe change
    (4, '\uFB3B') : (0,'vk'), #kaf
    (4, '\uFB3A') : (0,'vk'), #kaf sofit
    (4, '\u05DC') : (0,'vl'), #lamed
    (4, '\u05DE') : (0,'vm'), #mem
    (4, '\u05DD') : (0,'vm'), #mem sofit
    (4, '\u05E0') : (0,'vn'), #nun
    (4, '\u05DF') : (0,'vn'), #nun sofit
    (4, '\u05E1') : (0,'vs'), #samech
    (4, '\u05E4') : (7,'v'), #feh
    (4, '\u05E3') : (7,'v'), #feh sofit
    (4, '\uFB44') : (0,'vp'), #peh
    (4, '\uFB43') : (0,'vp'), #peh sofit
    (4, '\u05E6') : (0,'vts'), #tzadi
    (4, '\u05E5') : (0,'vts'), #tzadi sofit
    (4, '\u05E7') : (0,'vq'), #kaf
    (4, '\u05E8') : (0,'vr'), #resh
    (4, '\u05E9') : (3,'vs'), #shin
    (4, '\uFB2A') : (0,'vs'), #shin
    (4, '\uFB2B') : (0,'vs'), #sin
    (4, '\u05EA') : (0,'vt'), #tav
    (4, '\u05B7') : (0,'va'), #patach
    (4, '\u05B8') : (0,'va'), #kamatz
    (4, '\u05B2') : (0,'va'), #hataph patach
    (4, '\u05B1') : (0,'ve'), #hataph segol
    (4, '\u05B6') : (0,'ve'), #segol
    (4, '\u05B5') : (0,'ve'), #tzerei
    (4, '\u05B4') : (2,'vi'), #chiriq
    (4, '\u05BB') : (0,'vu'), #kubutz
    (4, '\u05B0') : (0,'v\u05B0'), #shva
    (4, '\u05B3') : (0,'vo'), #hataph kamatz
    (4, '\u05B9') : (0,'vo'), #holam chaser
    
    #stuff to deal with vav
    #assume vav unless holma to make it o or dagesh to make it u
    (5, '\u05D0') : (0,'v'), #aleph
    (5, '\u05E2') : (0,'v'), #ayin
    (5, '\u05D1') : (4,'v'), #bet
    (5, '\uFB31') : (0,'vb'), #vet
    (5, '\u05D2') : (0,'vg'), #gimmel
    (5, '\u05D3') : (0,'vd'), #daled
    (5, '\u05D4') : (0,'vh'), #hey
    (5, '\u05D5') : (5,''), #vav
    (5, '\uFB35') : (0,'vu'), #vav - shuruk
    (5, '\uFB4B') : (0,'vo'), #vav - holam
    (5, '\u05D6') : (0,'vz'), #zayin
    (5, '\u05D7') : (0,'vch'), #chet - maybe change
    (5, '\u05D8') : (0,'vt'), #tet
    (5, '\u05D9') : (0,'vy'), #youd
    (5, '\u05DB') : (6,'v'), #chaf - maybe change
    (5, '\u05DA') : (6,'v'), #chaf sofit - maybe change
    (5, '\uFB3B') : (0,'vk'), #kaf
    (5, '\uFB3A') : (0,'vk'), #kaf sofit
    (5, '\u05DC') : (0,'vl'), #lamed
    (5, '\u05DE') : (0,'vm'), #mem
    (5, '\u05DD') : (0,'vm'), #mem sofit
    (5, '\u05E0') : (0,'vn'), #nun
    (5, '\u05DF') : (0,'vn'), #nun sofit
    (5, '\u05E1') : (0,'vs'), #samech
    (5, '\u05E4') : (7,'v'), #feh
    (5, '\u05E3') : (7,'v'), #feh sofit
    (5, '\uFB44') : (0,'vp'), #peh
    (5, '\uFB43') : (0,'vp'), #peh sofit
    (5, '\u05E6') : (0,'vts'), #tzadi
    (5, '\u05E5') : (0,'vts'), #tzadi sofit
    (5, '\u05E7') : (0,'vq'), #kaf
    (5, '\u05E8') : (0,'vr'), #resh
    (5, '\u05E9') : (3,'vs'), #shin
    (5, '\uFB2A') : (0,'vs'), #shin
    (5, '\uFB2B') : (0,'vs'), #sin
    (5, '\u05EA') : (0,'vt'), #tav
    (5, '\u05B7') : (0,'va'), #patach
    (5, '\u05B8') : (0,'va'), #kamatz
    (5, '\u05B2') : (0,'va'), #hataph patach
    (5, '\u05B1') : (0,'ve'), #hataph segol
    (5, '\u05B6') : (0,'ve'), #segol
    (5, '\u05B5') : (0,'ve'), #tzerei
    (5, '\u05B4') : (2,'vi'), #chiriq
    (5, '\u05BB') : (0,'vu'), #kubutz
    (5, '\u05B0') : (0,'ve'), #shva
    (5, '\u05B3') : (0,'vo'), #hataph kamatz
    (5, '\u05B9') : (0,'o'), #holam chaser
    
     #stuff to deal with vav - dagesh
    #basicallly the cases where its a u not a v, but if theres another vowel there its a v
    (1, '\u05D0') : (0,'u'), #aleph
    (1, '\u05E2') : (0,'u'), #ayin
    (1, '\u05D1') : (4,'u'), #bet
    (1, '\uFB31') : (0,'ub'), #vet
    (1, '\u05D2') : (0,'ug'), #gimmel
    (1, '\u05D3') : (0,'ud'), #daled
    (1, '\u05D4') : (0,'uh'), #hey
    (1, '\u05D5') : (5,''), #vav
    (1, '\u05BA') : (0,'vo'), #vav - shuruk
    (1, '\u05B9') : (0,'vo'), #vav - holam
    (1, '\u05D6') : (0,'uz'), #zayin
    (1, '\u05D7') : (0,'uch'), #chet - maybe change
    (1, '\u05D8') : (0,'ut'), #tet
    (1, '\u05D9') : (0,'uy'), #youd
    (1, '\u05DB') : (6,'u'), #chaf - maybe change
    (1, '\u05DA') : (6,'u'), #chaf sofit - maybe change
    (1, '\uFB3B') : (0,'uk'), #kaf
    (1, '\uFB3A') : (0,'uk'), #kaf sofit
    (1, '\u05DC') : (0,'ul'), #lamed
    (1, '\u05DE') : (0,'um'), #mem
    (1, '\u05DD') : (0,'um'), #mem sofit
    (1, '\u05E0') : (0,'un'), #nun
    (1, '\u05DF') : (0,'un'), #nun sofit
    (1, '\u05E1') : (0,'us'), #samech
    (1, '\u05E4') : (7,'u'), #feh
    (1, '\u05E3') : (7,'u'), #feh sofit
    (1, '\uFB44') : (0,'up'), #peh
    (1, '\uFB43') : (0,'up'), #peh sofit
    (1, '\u05E6') : (0,'uts'), #tzadi
    (1, '\u05E5') : (0,'uts'), #tzadi sofit
    (1, '\u05E7') : (0,'uq'), #kaf
    (1, '\u05E8') : (0,'ur'), #resh
    (1, '\u05E9') : (3,'us'), #shin
    (1, '\uFB2A') : (0,'us'), #shin
    (1, '\uFB2B') : (0,'us'), #sin
    (1, '\u05EA') : (0,'ut'), #tav
    (1, '\u05B7') : (0,'va'), #patach
    (1, '\u05B8') : (0,'va'), #kamatz
    (1, '\u05B2') : (0,'va'), #hataph patach
    (1, '\u05B1') : (0,'ve'), #hataph segol
    (1, '\u05B6') : (0,'ve'), #segol
    (1, '\u05B5') : (0,'ve'), #tzerei
    (1, '\u05B4') : (2,'vi'), #chiriq
    (1, '\u05BB') : (0,'vu'), #kubutz
    (1, '\u05B0') : (0,'ve'), #shva
    (1, '\u05B3') : (0,'vo'), #hataph kamatz
    (1, '\u05B9') : (0,'vo'), #holam chaser
    
    
    #chaf kaf
    #assume chaf unless followed by a dagesh
    (6, '\u05D0') : (0,'ch'), #aleph
    (6, '\u05E2') : (0,'ch'), #ayin
    (6, '\u05D1') : (4,'ch'), #vet
    (6, '\u05D2') : (0,'chg'), #gimmel
    (6, '\u05D3') : (0,'chd'), #daled
    (6, '\u05D4') : (0,'chh'), #hey
    (6, '\u05D5') : (5,'ch'), #vav
    (6, '\u05D6') : (0,'chz'), #zayin
    (6, '\u05D7') : (0,'chch'), #chet - maybe change
    (6, '\u05D8') : (0,'cht'), #tet
    (6, '\u05D9') : (0,'chy'), #youd
    (6, '\u05DB') : (6,''), #chaf - maybe change
    (6, '\u05DA') : (6,''), #chaf sofit - maybe change
    (6, '\uFB3B') : (0,'chk'), #kaf
    (6, '\uFB3A') : (0,'chk'), #kaf sofit
    (6, '\u05DC') : (0,'chl'), #lamed
    (6, '\u05DE') : (0,'chm'), #mem
    (6, '\u05DD') : (0,'chm'), #mem sofit
    (6, '\u05E0') : (0,'chn'), #nun
    (6, '\u05DF') : (0,'chn'), #nun sofit
    (6, '\u05E1') : (0,'chs'), #samech
    (6, '\u05E4') : (7,'ch'), #feh
    (6, '\u05E3') : (7,'ch'), #feh sofit
    (6, '\uFB44') : (0,'chp'), #peh
    (6, '\uFB43') : (0,'chp'), #peh sofit
    (6, '\u05E6') : (0,'chts'), #tzadi
    (6, '\u05E5') : (0,'chts'), #tzadi sofit
    (6, '\u05E7') : (0,'chq'), #kaf
    (6, '\u05E8') : (0,'chr'), #resh
    (6, '\u05E9') : (3,'chs'), #shin
    (6, '\u05EA') : (0,'cht'), #tav
    (6, '\u05B7') : (0,'cha'), #patach
    (6, '\u05B8') : (0,'cha'), #kamatz
    (6, '\u05B2') : (0,'cha'), #hataph patach
    (6, '\u05B1') : (0,'che'), #hataph segol
    (6, '\u05B6') : (0,'che'), #segol
    (6, '\u05B5') : (0,'che'), #tzerei
    (6, '\u05B4') : (2,'chi'), #chiriq
    (6, '\u05BB') : (0,'chu'), #kubutz
    (6, '\u05B0') : (0,'ch\u05B0'), #shva
    (6, '\u05B3') : (0,'cho'), #hataph kamatz
    (6, '\u05B9') : (0,'cho'), #holam chaser
    
    #peh feh
    #assume feh unless followed by a peh
    (7, '\u05D0') : (0,'f'), #aleph
    (7, '\u05E2') : (0,'f'), #ayin
    (7, '\u05D1') : (4,'f'), #vet
    (7, '\u05D2') : (0,'fg'), #gimmel
    (7, '\u05D3') : (0,'fd'), #daled
    (7, '\u05D4') : (0,'fh'), #hey
    (7, '\u05D5') : (5,'fh'), #vav
    (7, '\u05D6') : (0,'fz'), #zayin
    (7, '\u05D7') : (0,'fch'), #chet - maybe change
    (7, '\u05D8') : (0,'ft'), #tet
    (7, '\u05D9') : (0,'fy'), #youd
    (7, '\u05DB') : (6,'f'), #chaf - maybe change
    (7, '\u05DA') : (6,'f'), #chaf sofit - maybe change
    (7, '\uFB3B') : (0,'fk'), #kaf
    (7, '\uFB3A') : (0,'fk'), #kaf sofit
    (7, '\u05DC') : (0,'fl'), #lamed
    (7, '\u05DE') : (0,'fm'), #mem
    (7, '\u05DD') : (0,'fm'), #mem sofit
    (7, '\u05E0') : (0,'fn'), #nun
    (7, '\u05DF') : (0,'fn'), #nun sofit
    (7, '\u05E1') : (0,'fs'), #samech
    (7, '\u05E4') : (7,''), #feh
    (7, '\u05E3') : (7,''), #feh sofit
    (7, '\uFB44') : (0,'fp'), #peh
    (7, '\uFB43') : (0,'fp'), #peh sofit
    (7, '\u05E6') : (0,'fts'), #tzadi
    (7, '\u05E5') : (0,'fts'), #tzadi sofit
    (7, '\u05E7') : (0,'fq'), #kaf
    (7, '\u05E8') : (0,'fr'), #resh
    (7, '\u05E9') : (3,'fs'), #shin
    (7, '\u05EA') : (0,'ft'), #tav
    (7, '\u05B7') : (0,'fa'), #patach
    (7, '\u05B8') : (0,'fa'), #kamatz
    (7, '\u05B2') : (0,'fa'), #hataph patach
    (7, '\u05B1') : (0,'fe'), #hataph segol
    (7, '\u05B6') : (0,'fe'), #segol
    (7, '\u05B5') : (0,'fe'), #tzerei
    (7, '\u05B4') : (2,'fi'), #chiriq
    (7, '\u05BB') : (0,'fu'), #kubutz
    (7, '\u05B0') : (0,'f\u05B0'), #shva
    (7, '\u05B3') : (0,'fo'), #hataph kamatz
    (7, '\u05B9') : (0,'fo'), #holam chaser
    
    (0, '\u05Bc') : (0,''), #dagesh
    (2, '\u05Bc') : (2,''), #dagesh
    (3, '\u05Bc') : (3,''), #dagesh
    (0, '@') : (0,'@'), #space
    
    (1, ' ') : (0,'u '), #space
    (3, ' ') : (0,' '), #space
    (4, ' ') : (0,'v '), #space
    (5, ' ') : (0,'v '), #space
    (6, ' ') : (0,'ch '), #space
    (7, ' ') : (0,'f '), #space
    
    (0,''):(True,''),
    (1,''):(True,''),
    (2,''):(True,''),
    (3,''):(True,'h'),
    (4,''):(True,'v'),
    (5,''):(True,'v'),
    (6,''):(True,'ch'),
    (7,''):(True,'f')
}



In [3]:
#First preprocess function. Removes teamim (cantilation marks) that are there for instructions 
#on chanting not on pronounciation
teamim = r"(?i)([\u0591-\u05AF]|[\u05c0]|[\u05BD])"

def preprocess_tanach(s):
    if re.search(teamim, s):
        s=re.sub(teamim, '',s)
        if re.search(teamim, s):
            print("here")
            return preprocess_tanach(s)
        else:
            return s
    return s

In [4]:
#second preprocess, makes sure that all diacritics are in the order the fst needs them 
#specifically makes sure that the dagesh, shuruk, mapiq and shin/sin dots come before the vowels

vowel = r"(?i)([\u05B0-\u05BB]|[\u05BD-\u05BF])"
vowel_and_dagesh = r"(?i)([\u05B0-\u05B8]|[\u05BD-\u05BF]|[\u05BB])([\u05BC]|[\u05c1]|[\u05c2])"
dagesh = r"(?i)([\u05BC]|[\u05c1]|[\u05c2])"

def preprocess(s):
    search = re.search(vowel_and_dagesh, s)
    if search:
        span=search.span()
        lists = (list(span))
        #The sub thing is a little weird because I had to split it up that was it didn't sub with
        #the same vowel across all of the places I needed to switch. So I split it up into 3 pieces
        #and only subbed in the middle, that way I can sub with diff vowels in diff places
        vowel_place = lists[0]
        dagesh_place = lists[1]
        search = list(search.group())
        vowel_in_string = search[0]
        dagesh_in_string =search[1]
        string_start = s[:vowel_place]
        string_middle = s[vowel_place:dagesh_place]
        string_end =s[dagesh_place:]
        string_middle = re.sub(vowel_and_dagesh, dagesh_in_string+vowel_in_string, string_middle)
        replaced = string_start + string_middle + string_end
        if re.search(vowel_and_dagesh, replaced):
            return preprocess(replaced)
        else:
            return replaced
    else:
        return s
    

In [5]:
#Postprocessor that inserts an apostrophe whenever two vowels come in a row 
#(call it after postprocess_shva)
#Same weird sub split as postprocess_tanach
vowels = r"(a|e|i|o|u)(a|e|i|o|u)"

def postprocess(s):
    search = re.search(vowels, s)
    if search:
        span=search.span()
        lists = (list(span))
        one_place = lists[0]
        two_place = lists[1]
        search = list(search.group())
        one_in_string = search[0]
        two_in_string =search[1]
        string_start = s[:one_place]
        string_middle = s[one_place:two_place]
        string_end =s[two_place:]
        string_middle = re.sub(vowels, one_in_string+"'"+two_in_string, string_middle)
        replaced = string_start + string_middle + string_end
        if re.search(vowels, replaced):
            return postprocess(replaced)
        else:
            return replaced
    else:
        return s

In [6]:
#postprocess that changes shva vowel to nothing or an apostrophe
#If the shva comes at the start of a word its a shva na (' or e sound)
#If a shva comes at the end of a word its a shva nach (silent)
#If two come in a row the first is a shva nach, the second is a shva na
#I literally had to look up these rules even though I speak the language

shva_na_start = r"(\b|^)[a-zA-Z][\u05B0]"
double_shva = r"[\u05B0][a-zA-Z][\u05B0]"
shva = r"\u05B0"
#Same weird sub split as postprocess_tanach
def postprocess_shva(s):
    search = re.search(shva_na_start, s)
    if search:
        span=search.span()
        lists = (list(span))
        one_place = lists[0]
        two_place = lists[1]
        search = list(search.group())
        one_in_string = search[0]
        two_in_string =search[1]
        string_start = s[:one_place]
        string_middle = s[one_place:two_place]
        string_end =s[two_place:]
        letter = search[0]
        string_middle = re.sub(shva_na_start, letter+"'", string_middle)
        s = string_start + string_middle + string_end
        if re.search(shva_na_start, s):
            return postprocess_shva(s)
    search_two = re.search(double_shva, s)
    if search_two:
        span=search_two.span()
        lists_two = (list(span))
        one_place = lists_two[0]
        two_place = lists_two[1]
        search_two = list(search_two.group())
        one_in_string = search_two[0]
        two_in_string =search_two[1]
        string_start = s[:one_place]
        string_middle = s[one_place:two_place]
        string_end =s[two_place:]
        letter = search_two[1]
        string_middle = re.sub(double_shva, letter+"'", string_middle)
        s = string_start + string_middle + string_end
        if re.search(double_shva, s):
            return postprocess_shva(s)
    search_three = re.search(shva, s)
    if search_three:
        replaced_three = re.sub(shva,'',s)
        return replaced_three
    else:
        return s


In [7]:
#function so you dont have to use all ten thousand processors seperately
#calls both preprocessors, then transduce with the fst, then the post processors

def full_use(s):
    s = postprocess(postprocess_shva(n_transduce(hebrew_to_english,preprocess(preprocess_tanach(s)))))
    return s


In [16]:
#Names and sentences for you to run as examples
#names:
print(full_use("דָּוִד בֶּן-גּוּרִיּוֹן"))
print(full_use("אֲרִיאֵל הַבְּרמַן")) #my last name is pronounced slightly differently in hebrew than in english
#bible verses
print(full_use("בְּרֵאשִׁ֖ית בָּרָ֣א אֱלֹהִ֑ים אֵ֥ת הַשָּׁמַ֖יִם וְאֵ֥ת הָאָֽרֶץ")) #Genesis 1:1
print(full_use("וַֽיִּרְעֲצ֤וּ וַיְרֹֽצְצוּ֙ אֶת־בְּנֵ֣י יִשְׂרָאֵ֔ל בַּשָּׁנָ֖ה הַהִ֑יא שְׁמֹנֶ֨ה עֶשְׂרֵ֜ה שָׁנָ֗ה אֶֽת־כָּל־בְּנֵ֤י יִשְׂרָאֵל֙ אֲשֶׁר֙ בְּעֵ֣בֶר הַיַּרְדֵּ֔ן בְּאֶ֥רֶץ הָאֱמֹרִ֖י אֲשֶׁ֥ר בַּגִּלְעָֽד")) #Judges 10:8
print(full_use("וַיֹּֽאמְרוּ֙ אֶל־מֹשֶׁ֔ה דַּבֵּר־אַתָּ֥ה עִמָּ֖נוּ וְנִשְׁמָ֑עָה וְאַל־יְדַבֵּ֥ר עִמָּ֛נוּ אֱלֹהִ֖ים פֶּן־נָמֽוּת")) #Exodus 20:16
print(full_use("וַיְהִ֣י ׀ בַּיּ֣וֹם הַשְּׁלִישִׁ֗י וַתִּלְבַּ֤שׁ אֶסְתֵּר֙ מַלְכ֔וּת וַֽתַּעֲמֹ֞ד בַּחֲצַ֤ר בֵּית־הַמֶּ֙לֶךְ֙ הַפְּנִימִ֔ית נֹ֖כַח בֵּ֣ית הַמֶּ֑לֶךְ וְ֠הַמֶּלֶךְ יוֹשֵׁ֞ב עַל־כִּסֵּ֤א מַלְכוּתוֹ֙ בְּבֵ֣ית הַמַּלְכ֔וּת נֹ֖כַח פֶּ֥תַח הַבָּֽיִת")) #Esther 5:1


david ben-guri'on
ari'el habrman
b'reshit bara elohim et hashamayim ve'et ha'arets
vayiratsu vayrotstsu et־b'ney yisra'el bashanah hahi shmoneh esreh shanah et־kal־b'ney yisra'el asher b'ever hayarden b'erets ha'emori asher bagilad
vayomru el־mosheh daber־atah imanu venishma'ah ve'al־y'daber imanu elohim pen־namut
vayhi  bayom hashlishi vatilbash ester malchut vata'amod bachatsar beyt־hamelech hapnimit nochach beyt hamelech vehamelech yoshev al־kise malchuto b'veyt hamalchut nochach petach habayit


In [9]:
#Not fully fleshed out fst for making it sound more like old hebrew transliterations 
#bet is never vet, y becomes j, p is always p never f...
old_hebrew_to_english = {
    
    (0, '\u05D0') : (0,''), #aleph
    (0, '\u05E2') : (0,''), #ayin
    
    (0, '\u05D1') : (0,'b'), #vet
    (0, '\uFB31') : (0,'b'), #bet
    
    (0, '\u05D2') : (0,'g'), #gimmel
    (0, '\u05D3') : (0,'d'), #daled
    (0, '\u05D4') : (0,'h'), #hey
    
    (0, '\u05D5') : (5,''), #vav
    (0, '\uFB35') : (0,'u'), #vav - shuruk
    (0, '\uFB4B') : (0,'o'), #vav - holam
    (5, '\u05BC') : (1,''), #dagesh
    (5, '\u05B9') : (0,'o'), #holam
    (5, '\u05BA') : (0,'o'), #holam haser
    
    (0, '\u05D6') : (0,'z'), #zayin
    (0, '\u05D7') : (0,'ch'), #chet - maybe change
    (0, '\u05D8') : (0,'t'), #tet
    (0, '\u05D9') : (0,'j'), #youd
    
    (0, '\u05DB') : (6,''), #chaf - maybe change
    (0, '\u05DA') : (6,''), #chaf sofit - maybe change
    (0, '\uFB3B') : (0,'k'), #kaf
    (0, '\uFB3A') : (0,'k'), #kaf sofit
    (6, '\u05BC') : (0,'k'), #dagesh
    
    (0, '\u05DC') : (0,'l'), #lamed
    
    (0, '\u05DE') : (0,'m'), #mem
    (0, '\u05DD') : (0,'m'), #mem sofit
    (0, '\u05E0') : (0,'n'), #nun
    (0, '\u05DF') : (0,'n'), #nun sofit
    
    (0, '\u05E1') : (0,'s'), #samech
    
    (0, '\u05E4') : (0,'p'), #feh
    (0, '\u05E3') : (0,'p'), #feh sofit
    (0, '\uFB44') : (0,'p'), #peh
    (0, '\uFB43') : (0,'p'), #peh sofit
    
    (0, '\u05E6') : (0,'ts'), #tzadi
    (0, '\u05E5') : (0,'ts'), #tzadi sofit
    
    (0, '\u05E7') : (0,'q'), #kaf
    (0, '\u05E8') : (0,'r'), #resh
    
    (0, '\u05E9') : (3,'s'), #shin
    (0, '\uFB2A') : (0,'s'), #shin
    (0, '\uFB2B') : (0,'s'), #sin
    (3, '\u05C1') : (0,'h'), #shin
    (3, '\u05C2') : (0,''), #shin
    
    (0, '\u05EA') : (0,'t'), #tav
    
    (0, '\u05B7') : (0,'a'), #patach
    (0, '\u05B8') : (0,'a'), #kamatz
    (0, '\u05B2') : (0,'a'), #hataph patach
    
    (0, '\u05B1') : (0,'e'), #hataph segol
    (0, '\u05B6') : (0,'e'), #segol
    (0, '\u05B5') : (0,'e'), #tzerei
    
    (0, '\u05B4') : (2,'i'), #chiriq
    (0, '\u05BB') : (0,'u'), #kubutz
    (0, '\u05B0') : (0,'\u05B0'), #shva
    
    (0, '\u05B3') : (0,'o'), #hataph kamatz
    (0, '\u05B9') : (0,'o'), #holam chaser
    
    #stuff to deal with chiriq youd
    #(if a youd has a tzerei vowel before it we just want an i not yi)
    (2, '\u05D9') : (0,''), #youd
    (2, '\u05D0') : (0,''), #aleph
    (2, '\u05E2') : (0,''), #ayin
    (2, '\u05D1') : (0,'b'), #bet
    (2, '\uFB31') : (0,'b'), #vet
    (2, '\u05D2') : (0,'g'), #gimmel
    (2, '\u05D3') : (0,'d'), #daled
    (2, '\u05D4') : (0,'h'), #hey
    (2, '\u05D5') : (5,''), #vav
    (2, '\uFB35') : (0,'u'), #vav - shuruk
    (2, '\uFB4B') : (0,'o'), #vav - holam
    (2, '\u05D6') : (0,'z'), #zayin
    (2, '\u05D7') : (0,'ch'), #chet - maybe change
    (2, '\u05D8') : (0,'t'), #tet
    (2, '\u05DB') : (6,''), #chaf - maybe change
    (2, '\u05DA') : (6,''), #chaf sofit - maybe change
    (2, '\uFB3B') : (0,'k'), #kaf
    (2, '\uFB3A') : (0,'k'), #kaf sofit
    (2, '\u05DC') : (0,'l'), #lamed
    (2, '\u05DE') : (0,'m'), #mem
    (2, '\u05DD') : (0,'m'), #mem sofit
    (2, '\u05E0') : (0,'n'), #nun
    (2, '\u05DF') : (0,'n'), #nun sofit
    (2, '\u05E1') : (0,'s'), #samech
    (2, '\u05E4') : (0,'p'), #feh
    (2, '\u05E3') : (0,'p'), #feh sofit
    (2, '\uFB44') : (0,'p'), #peh
    (2, '\uFB43') : (0,'p'), #peh sofit
    (2, '\u05E6') : (0,'ts'), #tzadi
    (2, '\u05E5') : (0,'ts'), #tzadi sofit
    (2, '\u05E7') : (0,'q'), #kaf
    (2, '\u05E8') : (0,'r'), #resh
    (2, '\u05E9') : (3,'s'), #shin
    (2, '\uFB2A') : (0,'s'), #shin
    (2, '\uFB2B') : (0,'s'), #sin
    (2, '\u05EA') : (0,'t'), #tav
    
    #stuff to deal with shin-sin
    #assume the letter is a sin if not follwed by the shin dot
    (3, '\u05D0') : (0,''), #aleph
    (3, '\u05E2') : (0,''), #ayin
    (3, '\u05D1') : (0,'b'), #bet
    (3, '\uFB31') : (0,'b'), #vet
    (3, '\u05D2') : (0,'g'), #gimmel
    (3, '\u05D3') : (0,'d'), #daled
    (3, '\u05D4') : (0,'h'), #hey
    (3, '\u05D5') : (5,''), #vav
    (3, '\uFB35') : (0,'u'), #vav - shuruk
    (3, '\uFB4B') : (0,'o'), #vav - holam
    (3, '\u05D6') : (0,'z'), #zayin
    (3, '\u05D7') : (0,'ch'), #chet - maybe change
    (3, '\u05D8') : (0,'t'), #tet
    (3, '\u05D9') : (0,'j'), #youd
    (3, '\u05DB') : (6,''), #chaf - maybe change
    (3, '\u05DA') : (6,''), #chaf sofit - maybe change
    (3, '\uFB3B') : (0,'k'), #kaf
    (3, '\uFB3A') : (0,'k'), #kaf sofit
    (3, '\u05DC') : (0,'l'), #lamed
    (3, '\u05DE') : (0,'m'), #mem
    (3, '\u05DD') : (0,'m'), #mem sofit
    (3, '\u05E0') : (0,'n'), #nun
    (3, '\u05DF') : (0,'n'), #nun sofit
    (3, '\u05E1') : (0,'s'), #samech
    (3, '\u05E4') : (0,'p'), #feh
    (3, '\u05E3') : (0,'p'), #feh sofit
    (3, '\uFB44') : (0,'p'), #peh
    (3, '\uFB43') : (0,'p'), #peh sofit
    (3, '\u05E6') : (0,'ts'), #tzadi
    (3, '\u05E5') : (0,'ts'), #tzadi sofit
    (3, '\u05E7') : (0,'q'), #kaf
    (3, '\u05E8') : (0,'r'), #resh
    (3, '\u05E9') : (3,'s'), #shin
    (3, '\uFB2A') : (0,'s'), #shin
    (3, '\uFB2B') : (0,'s'), #sin
    (3, '\u05EA') : (0,'t'), #tav
    (3, '\u05B7') : (0,'a'), #patach
    (3, '\u05B8') : (0,'a'), #kamatz
    (3, '\u05B2') : (0,'a'), #hataph patach
    (3, '\u05B1') : (0,'e'), #hataph segol
    (3, '\u05B6') : (0,'e'), #segol
    (3, '\u05B5') : (0,'e'), #tzerei
    (3, '\u05B4') : (2,'i'), #chiriq
    (3, '\u05BB') : (0,'u'), #kubutz
    (3, '\u05B0') : (0,'\u05B0'), #shva
    (3, '\u05B3') : (0,'o'), #hataph kamatz
    (3, '\u05B9') : (0,'o'), #holam chaser
    
    
    #stuff to deal with vav
    #assume vav unless holma to make it o or dagesh to make it u
    (5, '\u05D0') : (0,'w'), #aleph
    (5, '\u05E2') : (0,'w'), #ayin
    (5, '\u05D1') : (0,'wb'), #bet
    (5, '\uFB31') : (0,'wb'), #vet
    (5, '\u05D2') : (0,'wg'), #gimmel
    (5, '\u05D3') : (0,'wd'), #daled
    (5, '\u05D4') : (0,'wh'), #hey
    (5, '\u05D5') : (5,''), #vav
    (5, '\uFB35') : (0,'wu'), #vav - shuruk
    (5, '\uFB4B') : (0,'wo'), #vav - holam
    (5, '\u05D6') : (0,'wz'), #zayin
    (5, '\u05D7') : (0,'wch'), #chet - maybe change
    (5, '\u05D8') : (0,'wt'), #tet
    (5, '\u05D9') : (0,'wj'), #youd
    (5, '\u05DB') : (6,'w'), #chaf - maybe change
    (5, '\u05DA') : (6,'w'), #chaf sofit - maybe change
    (5, '\uFB3B') : (0,'wk'), #kaf
    (5, '\uFB3A') : (0,'wk'), #kaf sofit
    (5, '\u05DC') : (0,'wl'), #lamed
    (5, '\u05DE') : (0,'wm'), #mem
    (5, '\u05DD') : (0,'wm'), #mem sofit
    (5, '\u05E0') : (0,'wn'), #nun
    (5, '\u05DF') : (0,'wn'), #nun sofit
    (5, '\u05E1') : (0,'ws'), #samech
    (5, '\u05E4') : (0,'wp'), #feh
    (5, '\u05E3') : (0,'wp'), #feh sofit
    (5, '\uFB44') : (0,'wp'), #peh
    (5, '\uFB43') : (0,'wp'), #peh sofit
    (5, '\u05E6') : (0,'wts'), #tzadi
    (5, '\u05E5') : (0,'wts'), #tzadi sofit
    (5, '\u05E7') : (0,'wq'), #kaf
    (5, '\u05E8') : (0,'wr'), #resh
    (5, '\u05E9') : (3,'ws'), #shin
    (5, '\uFB2A') : (0,'ws'), #shin
    (5, '\uFB2B') : (0,'ws'), #sin
    (5, '\u05EA') : (0,'wt'), #tav
    (5, '\u05B7') : (0,'wa'), #patach
    (5, '\u05B8') : (0,'wa'), #kamatz
    (5, '\u05B2') : (0,'wa'), #hataph patach
    (5, '\u05B1') : (0,'we'), #hataph segol
    (5, '\u05B6') : (0,'we'), #segol
    (5, '\u05B5') : (0,'we'), #tzerei
    (5, '\u05B4') : (2,'wi'), #chiriq
    (5, '\u05BB') : (0,'wu'), #kubutz
    (5, '\u05B0') : (0,'we'), #shva
    (5, '\u05B3') : (0,'wo'), #hataph kamatz
    (5, '\u05B9') : (0,'o'), #holam chaser
    
     #stuff to deal with vav
    #assume vav unless holma to make it o or dagesh to make it u
    (1, '\u05D0') : (0,'u'), #aleph
    (1, '\u05E2') : (0,'u'), #ayin
    (1, '\u05D1') : (0,'ub'), #bet
    (1, '\uFB31') : (0,'ub'), #vet
    (1, '\u05D2') : (0,'ug'), #gimmel
    (1, '\u05D3') : (0,'ud'), #daled
    (1, '\u05D4') : (0,'uh'), #hey
    (1, '\u05D5') : (5,''), #vav
    (1, '\u05BA') : (0,'wo'), #vav - shuruk
    (1, '\u05B9') : (0,'wo'), #vav - holam
    (1, '\u05D6') : (0,'uz'), #zayin
    (1, '\u05D7') : (0,'uch'), #chet - maybe change
    (1, '\u05D8') : (0,'ut'), #tet
    (1, '\u05D9') : (0,'uj'), #youd
    (1, '\u05DB') : (6,'u'), #chaf - maybe change
    (1, '\u05DA') : (6,''), #chaf sofit - maybe change
    (1, '\uFB3B') : (0,'uk'), #kaf
    (1, '\uFB3A') : (0,'uk'), #kaf sofit
    (1, '\u05DC') : (0,'ul'), #lamed
    (1, '\u05DE') : (0,'um'), #mem
    (1, '\u05DD') : (0,'um'), #mem sofit
    (1, '\u05E0') : (0,'un'), #nun
    (1, '\u05DF') : (0,'un'), #nun sofit
    (1, '\u05E1') : (0,'us'), #samech
    (1, '\u05E4') : (0,'up'), #feh
    (1, '\u05E3') : (0,'up'), #feh sofit
    (1, '\uFB44') : (0,'up'), #peh
    (1, '\uFB43') : (0,'up'), #peh sofit
    (1, '\u05E6') : (0,'uts'), #tzadi
    (1, '\u05E5') : (0,'uts'), #tzadi sofit
    (1, '\u05E7') : (0,'uq'), #kaf
    (1, '\u05E8') : (0,'ur'), #resh
    (1, '\u05E9') : (3,'us'), #shin
    (1, '\uFB2A') : (0,'us'), #shin
    (1, '\uFB2B') : (0,'us'), #sin
    (1, '\u05EA') : (0,'ut'), #tav
    (1, '\u05B7') : (0,'wa'), #patach
    (1, '\u05B8') : (0,'wa'), #kamatz
    (1, '\u05B2') : (0,'wa'), #hataph patach
    (1, '\u05B1') : (0,'we'), #hataph segol
    (1, '\u05B6') : (0,'we'), #segol
    (1, '\u05B5') : (0,'we'), #tzerei
    (1, '\u05B4') : (2,'wi'), #chiriq
    (1, '\u05BB') : (0,'wu'), #kubutz
    (1, '\u05B0') : (0,'we'), #shva
    (1, '\u05B3') : (0,'wo'), #hataph kamatz
    (1, '\u05B9') : (0,'wo'), #holam chaser
    
    
    #chaf kaf
    #assume chaf unless followed by a dagesh
    (6, '\u05D0') : (0,'ch'), #aleph
    (6, '\u05E2') : (0,'ch'), #ayin
    (6, '\u05D1') : (0,'chb'), #vet
    (6, '\u05D2') : (0,'chg'), #gimmel
    (6, '\u05D3') : (0,'chd'), #daled
    (6, '\u05D4') : (0,'chh'), #hey
    (6, '\u05D5') : (5,'ch'), #vav
    (6, '\u05D6') : (0,'chz'), #zayin
    (6, '\u05D7') : (0,'chch'), #chet - maybe change
    (6, '\u05D8') : (0,'cht'), #tet
    (6, '\u05D9') : (0,'chj'), #youd
    (6, '\u05DB') : (6,''), #chaf - maybe change
    (6, '\u05DA') : (6,''), #chaf sofit - maybe change
    (6, '\uFB3B') : (0,'chk'), #kaf
    (6, '\uFB3A') : (0,'chk'), #kaf sofit
    (6, '\u05DC') : (0,'chl'), #lamed
    (6, '\u05DE') : (0,'chm'), #mem
    (6, '\u05DD') : (0,'chm'), #mem sofit
    (6, '\u05E0') : (0,'chn'), #nun
    (6, '\u05DF') : (0,'chn'), #nun sofit
    (6, '\u05E1') : (0,'chs'), #samech
    (6, '\u05E4') : (0,'chp'), #feh
    (6, '\u05E3') : (0,'chp'), #feh sofit
    (6, '\uFB44') : (0,'chp'), #peh
    (6, '\uFB43') : (0,'chp'), #peh sofit
    (6, '\u05E6') : (0,'chts'), #tzadi
    (6, '\u05E5') : (0,'chts'), #tzadi sofit
    (6, '\u05E7') : (0,'chq'), #kaf
    (6, '\u05E8') : (0,'chr'), #resh
    (6, '\u05E9') : (3,'chs'), #shin
    (6, '\u05EA') : (0,'cht'), #tav
    (6, '\u05B7') : (0,'cha'), #patach
    (6, '\u05B8') : (0,'cha'), #kamatz
    (6, '\u05B2') : (0,'cha'), #hataph patach
    (6, '\u05B1') : (0,'che'), #hataph segol
    (6, '\u05B6') : (0,'che'), #segol
    (6, '\u05B5') : (0,'che'), #tzerei
    (6, '\u05B4') : (2,'chi'), #chiriq
    (6, '\u05BB') : (0,'chu'), #kubutz
    (6, '\u05B0') : (0,'ch\u05B0'), #shva
    (6, '\u05B3') : (0,'cho'), #hataph kamatz
    (6, '\u05B9') : (0,'cho'), #holam chaser
    
    
    (0, '\u05Bc') : (0,''), #dagesh
    (2, '\u05Bc') : (2,''), #dagesh
    (3, '\u05Bc') : (3,''), #dagesh
    (0, '@') : (0,'@'), #space
    
    (1, ' ') : (0,'u '), #space
    (3, ' ') : (0,' '), #space
    (4, ' ') : (0,'v '), #space
    (5, ' ') : (0,'v '), #space
    (6, ' ') : (0,'ch '), #space
    (7, ' ') : (0,'f '), #space
    
    (0,''):(True,''),
    (1,''):(True,''),
    (2,''):(True,''),
    (3,''):(True,'h'),
    (4,''):(True,'v'),
    (5,''):(True,'v'),
    (6,''):(True,'ch'),
    (7,''):(True,'f')
}

In [10]:
#Sources/Citations
#https://superuser.com/questions/29146/how-to-enter-hebrew-with-diacritics-nikkud-in-windows
#https://en.wikipedia.org/wiki/Shva
#https://en.wikipedia.org/wiki/ISO_259
#https://en.wikipedia.org/wiki/Unicode_and_HTML_for_the_Hebrew_alphabet
#https://docs.python.org/3/library/re.html#regular-expression-objects
#https://www.sefaria.org/texts